In [2]:
#import extract feature
import os
import librosa
import pandas as pd
import numpy as np
from tqdm import tqdm

# label encoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

#train test split
from sklearn.model_selection import train_test_split

#import model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam 
from sklearn import metrics
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import LayerNormalization
from sklearn.model_selection import cross_val_score
from tensorflow import keras
import keras

#import train model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

### Extracting MFCC's for every audio file
audio_dataset_path = 'ProjectSounds/audio'
metadata = pd.read_csv('ProjectSounds/metadata/SoundData.csv')
metadata.head()

def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)
    
    return mfccs_scaled_features
    
### Now we iterate through every audio file and extract features
## using Mel-Frequency Ceptral Coefficients
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    final_class_labels = row["class"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])
    
    
### Converting extracted_features to Pandas dataframe
extracted_features_df = pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

### Split the dataset into independent and dependent dataset
x = np.array(extracted_features_df['feature'].tolist())
y = np.array(extracted_features_df['class'].tolist())

labelencoder = LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

### number of classes
num_labels=y.shape[1]


197it [00:12, 16.22it/s]


In [57]:
def create_model():
    model=Sequential()

    model.add(Flatten(input_shape=(40,)))

    ###first layer
    model.add(Dense(100, Activation('sigmoid')))
    model.add(Dropout(0.5))

    ###second layer
    model.add(Dense(200, Activation('sigmoid')))
    model.add(Dropout(0.5))

    ###third layer
    model.add(Dense(100, Activation('sigmoid')))
    model.add(Dropout(0.5))

    ###final layer
    model.add(Dense(num_labels, Activation('softmax')))
    
    model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
    
    return model

In [58]:
model = create_model()

In [59]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_11 (Flatten)         (None, 40)                0         
_________________________________________________________________
dense_44 (Dense)             (None, 100)               4100      
_________________________________________________________________
dropout_33 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_45 (Dense)             (None, 200)               20200     
_________________________________________________________________
dropout_34 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_46 (Dense)             (None, 100)               20100     
_________________________________________________________________
dropout_35 (Dropout)         (None, 100)             

In [60]:
num_epochs = 100
num_batch_size = 32

checkpoint_path = "save_model/my_model.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, 
                                                 verbose=1, 
                                                 save_weights_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[cp_callback])


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/100
5/5 [==============================] - 1s 36ms/step - loss: 0.9490 - accuracy: 0.5287 - val_loss: 0.7026 - val_accuracy: 0.4500

Epoch 00001: saving model to save_model/my_model.ckpt
Epoch 2/100
5/5 [==============================] - 0s 6ms/step - loss: 0.8364 - accuracy: 0.5032 - val_loss: 0.7302 - val_accuracy: 0.5500

Epoch 00002: saving model to save_model/my_model.ckpt
Epoch 3/100
5/5 [==============================] - 0s 6ms/step - loss: 0.8231 - accuracy: 0.4968 - val_loss: 0.6860 - val_accuracy: 0.5500

Epoch 00003: saving model to save_model/my_model.ckpt
Epoch 4/100
5/5 [==============================] - 0s 5ms/step - loss: 0.7674 - accuracy: 0.4650 - val_loss: 0.7196 - val_accuracy: 0.4500

Epoch 00004: saving model to save_model/my_model.ckpt
Epoch 5/100
5/5 [==============================] - 0s 6ms/step - loss: 0.7885 - accuracy: 0.4904 - val_loss: 0.7346 - val_accuracy: 0.4500

Epoch 00005: saving model to save_model/my_model.ckpt
Epoch 6/100
5/5 [============

5/5 [==============================] - 0s 5ms/step - loss: 0.0624 - accuracy: 0.9809 - val_loss: 0.0064 - val_accuracy: 1.0000

Epoch 00043: saving model to save_model/my_model.ckpt
Epoch 44/100
5/5 [==============================] - 0s 5ms/step - loss: 0.0493 - accuracy: 0.9873 - val_loss: 0.0055 - val_accuracy: 1.0000

Epoch 00044: saving model to save_model/my_model.ckpt
Epoch 45/100
5/5 [==============================] - 0s 5ms/step - loss: 0.0504 - accuracy: 0.9936 - val_loss: 0.0042 - val_accuracy: 1.0000

Epoch 00045: saving model to save_model/my_model.ckpt
Epoch 46/100
5/5 [==============================] - 0s 5ms/step - loss: 0.0184 - accuracy: 1.0000 - val_loss: 0.0039 - val_accuracy: 1.0000

Epoch 00046: saving model to save_model/my_model.ckpt
Epoch 47/100
5/5 [==============================] - 0s 5ms/step - loss: 0.0361 - accuracy: 0.9936 - val_loss: 0.0036 - val_accuracy: 1.0000

Epoch 00047: saving model to save_model/my_model.ckpt
Epoch 48/100
5/5 [====================


Epoch 00084: saving model to save_model/my_model.ckpt
Epoch 85/100
5/5 [==============================] - 0s 6ms/step - loss: 0.0069 - accuracy: 1.0000 - val_loss: 5.0256e-04 - val_accuracy: 1.0000

Epoch 00085: saving model to save_model/my_model.ckpt
Epoch 86/100
5/5 [==============================] - 0s 5ms/step - loss: 0.0067 - accuracy: 1.0000 - val_loss: 4.8102e-04 - val_accuracy: 1.0000

Epoch 00086: saving model to save_model/my_model.ckpt
Epoch 87/100
5/5 [==============================] - 0s 5ms/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 4.6598e-04 - val_accuracy: 1.0000

Epoch 00087: saving model to save_model/my_model.ckpt
Epoch 88/100
5/5 [==============================] - 0s 5ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 4.5419e-04 - val_accuracy: 1.0000

Epoch 00088: saving model to save_model/my_model.ckpt
Epoch 89/100
5/5 [==============================] - 0s 5ms/step - loss: 0.0142 - accuracy: 0.9936 - val_loss: 4.3827e-04 - val_accuracy: 1.0000

Epoc

In [61]:
_model = KerasClassifier(build_fn = create_model, batch_size=32, epochs=100)
accuracies_test = cross_val_score(_model,x_test,y_test,cv=10,n_jobs=-1)
accuracies_train = cross_val_score(_model,x_train,y_train,cv=10,n_jobs=-1)

In [62]:
print("This is accuracies of cross validation by x_train from tanh: \n",accuracies_train)
print("----------------------------------------------------------------------------")
print("This is accuracies of cross validation by x_test from tanh: \n",accuracies_test)

This is accuracies of cross validation by x_train from tanh: 
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
----------------------------------------------------------------------------
This is accuracies of cross validation by x_test from tanh: 
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [63]:
os.listdir(checkpoint_dir)

['checkpoint', 'my_model.ckpt.index', 'my_model.ckpt.data-00000-of-00001']

In [64]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(x_test, y_test, verbose=0)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

Untrained model, accuracy: 55.00%


In [65]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(x_test, y_test, verbose=0)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

Restored model, accuracy: 100.00%
